In [30]:
import sqlite3
import pandas as pd
import time

st = time.time()

conn = sqlite3.connect("stock.db")
ds_ratio = pd.read_sql_query('select STOCK_CODE, STOCK_FACTOR, STOCK_YEAR, STOCK_DATA from VCSC_FINANCIAL_RATIO', conn)
ds_price = pd.read_sql_query('select STOCK_CODE, STOCK_DATE, STOCK_CLOSE from VCSC_STOCK_PRICE', conn)
conn.close()

print("1--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'] == 'EPS (VND)']
#ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'] == 'BVS']

ds_ratio = ds_ratio.loc[ds_ratio['STOCK_YEAR'].str.contains('TTM')==False]
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace('^-$', '')
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace(',', '')
ds_ratio['STOCK_DATA'] = pd.to_numeric(ds_ratio['STOCK_DATA'])
ds_ratio['STOCK_YEAR'] = pd.to_numeric(ds_ratio['STOCK_YEAR'])

print("2--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price['STOCK_DATE'] = pd.to_datetime(ds_price['STOCK_DATE'], format='%m/%d/%Y')
ds_price['STOCK_YEAR'] = ds_price['STOCK_DATE'].dt.year
ds_price['STOCK_QUARTER'] = ds_price['STOCK_YEAR'].astype('str') + ds_price['STOCK_DATE'].dt.quarter.astype('str')
ds_price['STOCK_CLOSE'] = ds_price['STOCK_CLOSE'].str.replace(',', '').astype(float)

print("3--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_QUARTER']).nth(0)
ds_price['NEXT-PRICE-Q'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)
ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_YEAR']).nth(0)
ds_price['NEXT-PRICE-Y'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)

print("4--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_ratio = ds_ratio.merge(ds_price, left_on=['STOCK_CODE', 'STOCK_YEAR'], right_on=['STOCK_CODE', 'STOCK_YEAR'], how='right')
ds_ratio['EP'] = ds_ratio['STOCK_DATA'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio = ds_ratio.loc[ds_ratio['EP'].notnull()]
ds_ratio['RETURN'] = ds_ratio['NEXT-PRICE-Y'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['DECILE'] = (ds_ratio.groupby('STOCK_YEAR')['EP']
                                .apply(lambda x: pd.qcut(x,10,duplicates='drop',labels=False)))

s1 = ds_ratio.groupby(['STOCK_YEAR','DECILE'], as_index=False).mean()

print("5--- %s seconds ---" % (time.time() - st))
st = time.time()

s1.pivot(index='DECILE', columns='STOCK_YEAR', values='RETURN')

1--- 3.007172107696533 seconds ---
2--- 0.21601247787475586 seconds ---
3--- 7.3564207553863525 seconds ---
4--- 1.3730785846710205 seconds ---
5--- 0.13200736045837402 seconds ---


STOCK_YEAR,2011,2012,2013,2014,2015,2016,2017
DECILE,,,,,,,
0,0.821977,1.365853,0.922564,0.962444,1.073433,1.162584,0.904790
1,0.873232,1.136238,0.985236,0.940798,0.867356,1.166750,0.905766
2,0.875951,1.092542,1.157756,1.037609,1.098093,1.078077,0.920267
3,0.964071,1.163420,1.016193,1.109804,1.023630,1.195339,0.910185
4,1.010218,1.246399,1.012645,1.091648,1.074642,1.079726,0.939526
5,1.148225,1.262236,1.046444,1.143598,1.108959,1.147481,0.931729
6,1.036380,1.301958,1.106507,1.266797,1.121440,1.331608,0.971727
7,1.134772,1.371661,1.143041,1.138840,1.132009,1.083792,0.968063
8,1.189487,1.379031,1.185880,1.150396,1.112128,1.078803,0.962366
